# Annotate variants in VCF file with JSON fields

In [1]:
import os
os.environ['PYSPARK_PYTHON'] = 'python3'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3'
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://bbench.tst-aws-ilmn-integration-w513952.tst-aws-ilmn-integration-w513952.svc.cluster.local:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.63-295f119ef95b
LOGGING: writing to /data/GRE/gre_hail/hail-20210308-1840-0.2.63-295f119ef95b.log


The VCF file, written to '/tmp/dummy.vcf'

In [2]:
vcf_string = '''##fileformat=VCFv4.2
#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\tFORMAT\t3344897\t1594378
chr11\t65918810\tchr11_65918810_C_G\tC\tG,A\t40\t.\tAF=2e-06;AQ=40\tGT:DP:AD:GQ:PL:RNC\t0/0:20:20,0:50:0,69,689:..\t0/0:18:18,0:48:0,54,539:..
chr11\t65918812\tchr11_65918812_G_A\tG\tA\t49\t.\tAF=5e-06;AQ=49\tGT:DP:AD:GQ:PL:RNC\t0/0:20:20,0:50:0,69,689:..\t0/0:18:18,0:48:0,54,539:..'''
with open('/tmp/dummy.vcf','wt') as out:
    out.write(vcf_string)

Import into HAIL

In [3]:
mt = hl.import_vcf(
    '/tmp/dummy.vcf',
    reference_genome="GRCh38",
    array_elements_required=False
)
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {}
----------------------------------------
Entry fields:
    None
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


The Annotation JSON, written to '/tmp/annotations.json' as NewlineDelimitedJSON (NDJSON)

In [4]:
ndjson = '''{"chromosome": "chr11", "position": 65918810, "refAllele": "C", "altAlleles": ["G", "A"],"variants": [{"vid": "11-65918810-C-G", "hgvsg": "NC_000011.10:g.65918810C>G", "phylopScore": -0.4}, { "vid": "11-65918810-C-A", "hgvsg": "NC_000011.10:g.65918810C>A", "phylopScore": -0.8}]} 
{"chromosome": "chr11", "position": 65918812, "refAllele": "G", "altAlleles": ["A"], "variants": [{"vid": "11-65918812-G-A", "phylopScore": 0.1}]}
'''
with open('/tmp/annotations.json', 'wt') as out:
    out.write(ndjson)
ht = hl.import_table('/tmp/annotations.json',no_header=True)
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'f0': str 
----------------------------------------
Key: []
----------------------------------------


2021-03-08 18:40:41 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (not specified)


Describe the entries from the JSON file we want to use as annotation (In this case, it is a subset; the annotation "phylopScore" is deliberately left out.

This is using the "hail shortcut notation".

In [5]:
nirvana_schema = '''
struct{
    chromosome: str,
    position: int32,
    refAllele: str,
    altAlleles: array<str>,
    cytogeneticBand:str,
    variants:array<struct{
        vid:str,
        hgvsg:str
    }>
}
'''

Parse the NDJSON field from the table with ```hl.parse_json``` function, which results in a ```Hail Expression``` which we can use to extract the locus information we need as KEY.

In [6]:
json_expr = hl.parse_json(ht.f0,dtype=nirvana_schema)
json_expr.describe()

--------------------------------------------------------
Type:
        struct {
        chromosome: str, 
        position: int32, 
        refAllele: str, 
        altAlleles: array<str>, 
        cytogeneticBand: str, 
        variants: array<struct {
            vid: str, 
            hgvsg: str
        }>
    }
--------------------------------------------------------
Source:
Index:
    ['row']
--------------------------------------------------------


Create a new hail table using the locus, alleles as key and the nirvana structure as annotation field.

Using the ```hl.locus``` function to construct the locus from the chromosome and position information, extracted from the expression.

Using the ```hl.array``` function and the extend function to construct the alleles array, and using the complete json_expr as the nirvana annotation

In [7]:
ht = ht.key_by(
    locus = hl.locus(json_expr.chromosome,json_expr.position,reference_genome='GRCh38'),
    alleles = hl.array([json_expr.refAllele]).extend(json_expr.altAlleles),
    nirvana = json_expr
).drop('f0').key_by('locus','alleles')
ht.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Row fields:
    'locus': locus<GRCh38> 
    'alleles': array<str> 
    'nirvana': struct {
        chromosome: str, 
        position: int32, 
        refAllele: str, 
        altAlleles: array<str>, 
        cytogeneticBand: str, 
        variants: array<struct {
            vid: str, 
            hgvsg: str
        }>
    } 
----------------------------------------
Key: ['locus', 'alleles']
----------------------------------------


Now annotate the Hail MatrixTable with the new row annotation table.

We are using the ```**``` notation to ensure the annotation will use the name "nirvana". If we would have used the following notation:

```python
mt = mt.annotate_rows(nirvana = new_ht[mt.row_key])
```

The nirvana struct would be encapsulated inside another struct, called nirvana, like this:

    ----------------------------------------
    Global fields:
        None
    ----------------------------------------
    Column fields:
        's': str
    ----------------------------------------
    Row fields:
        'locus': locus<GRCh38>
        'alleles': array<str>
        'rsid': str
        'qual': float64
        'filters': set<str>
        'info': struct {}
        'nirvana': struct {
            nirvana: struct {
                chromosome: str, 
                position: int32, 
                refAllele: str, 
                altAlleles: array<str>, 
                cytogeneticBand: str, 
                variants: array<struct {
                    vid: str, 
                    hgvsg: str
                }>
            }
        }
    ----------------------------------------
    Entry fields:
        None
    ----------------------------------------
    Column key: ['s']
    Row key: ['locus', 'alleles']
    ----------------------------------------

In [8]:
mt = mt.annotate_rows(**ht[mt.row_key])
mt.describe()

----------------------------------------
Global fields:
    None
----------------------------------------
Column fields:
    's': str
----------------------------------------
Row fields:
    'locus': locus<GRCh38>
    'alleles': array<str>
    'rsid': str
    'qual': float64
    'filters': set<str>
    'info': struct {}
    'nirvana': struct {
        chromosome: str, 
        position: int32, 
        refAllele: str, 
        altAlleles: array<str>, 
        cytogeneticBand: str, 
        variants: array<struct {
            vid: str, 
            hgvsg: str
        }>
    }
----------------------------------------
Entry fields:
    None
----------------------------------------
Column key: ['s']
Row key: ['locus', 'alleles']
----------------------------------------


In [9]:
mt.rows().show()

2021-03-08 18:40:42 Hail: INFO: Coerced sorted dataset
2021-03-08 18:40:42 Hail: WARN: struct{vid: str, hgvsg: str} has no field phylopScore at <root>.variants[element] for value JDouble(-0.4)
2021-03-08 18:40:42 Hail: WARN: struct{vid: str, hgvsg: str} has no field phylopScore at <root>.variants[element] for value JDouble(0.1)
2021-03-08 18:40:42 Hail: INFO: Coerced sorted dataset
2021-03-08 18:40:43 Hail: WARN: struct{vid: str, hgvsg: str} has no field phylopScore at <root>.variants[element] for value JDouble(-0.4)
2021-03-08 18:40:43 Hail: WARN: struct{vid: str, hgvsg: str} has no field phylopScore at <root>.variants[element] for value JDouble(0.1)
2021-03-08 18:40:43 Hail: WARN: struct{vid: str, hgvsg: str} has no field phylopScore at <root>.variants[element] for value JDouble(-0.4)
2021-03-08 18:40:43 Hail: WARN: struct{vid: str, hgvsg: str} has no field phylopScore at <root>.variants[element] for value JDouble(0.1)


+----------------+---------------+----------------------+----------+----------+
| locus          | alleles       | rsid                 |     qual | filters  |
+----------------+---------------+----------------------+----------+----------+
| locus<GRCh38>  | array<str>    | str                  |  float64 | set<str> |
+----------------+---------------+----------------------+----------+----------+
| chr11:65918810 | ["C","G","A"] | "chr11_65918810_C_G" | 4.00e+01 | NA       |
| chr11:65918812 | ["G","A"]     | "chr11_65918812_G_A" | 4.90e+01 | NA       |
+----------------+---------------+----------------------+----------+----------+

+--------------------+------------------+-------------------+
| nirvana.chromosome | nirvana.position | nirvana.refAllele |
+--------------------+------------------+-------------------+
| str                |            int32 | str               |
+--------------------+------------------+-------------------+
| "chr11"            |         65918810 | "C"               |
| "chr11"            |         65918812 | "G"               |
+--------------------+------------------+-------------------+

+--------------------+-------------------------+
| nirvana.altAlleles | nirvana.cytogeneticBand |
+--------------------+-------------------------+
| array<str>         | str                     |
+--------------------+-------------------------+
| ["G","A"]          | NA                      |
| ["A"]              | NA                      |
+--------------------+-------------------------+

+------------------------------------------------------------------------------+
| nirvana.variants                                                             |
+------------------------------------------------------------------------------+
| array<struct{vid: str, hgvsg: str}>                                          |
+------------------------------------------------------------------------------+
| [("11-65918810-C-G","NC_000011.10:g.65918810C>G"),("11-65918810-C-A","NC_... |
| [("11-65918812-G-A",NA)]                                                     |
+------------------------------------------------------------------------------+